In [20]:
import pandas as pd
import numpy as np

from ta import momentum, trend, volume
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [21]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1MINUTE,
                                      str(3000) + ' minutes ago UTC'))

# First 6 column represent Time and OHLCV values
df = df.iloc[:, :6]
df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
df['Time'] = pd.to_datetime(df['Time'], unit='ms')
for column in df.columns:
    if column != 'Time':
        df[column] = df[column].astype(float)

# Set Time as index
df.set_index('Time', inplace=True)

# Generate Technical Indicators

In [22]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2024-07-15 17:58:00,63111.25,63166.67,63111.24,63126.67,17.06148
2024-07-15 17:59:00,63126.67,63126.67,63081.96,63081.97,21.89418
2024-07-15 18:00:00,63081.97,63095.28,63012.97,63032.62,71.30023
2024-07-15 18:01:00,63032.62,63070.01,62993.33,63070.01,53.44771
2024-07-15 18:02:00,63070.01,63136.69,63069.47,63124.08,32.53447
...,...,...,...,...,...
2024-07-17 19:53:00,64741.99,64748.00,64713.73,64713.74,10.98987
2024-07-17 19:54:00,64713.73,64739.05,64713.73,64720.97,14.63963
2024-07-17 19:55:00,64720.97,64720.97,64669.99,64683.47,13.46099


In [23]:
df['SMA50'] = df['Close'].rolling(50).mean()
df['SMA200'] = df['Close'].rolling(200).mean()
df['RSI'] = momentum.rsi(df['Close'])
df['MACD'] = trend.macd(df['Close'])
df['MACD_sig'] = trend.macd_signal(df['Close'])
df['MACD_diff'] = trend.macd_diff(df['Close'])
df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)
df['ADX'] = trend.adx(df['High'], df['Low'], df['Close'])
df['-DI'] = trend.adx_neg(df['High'], df['Low'], df['Close'])
df['+DI'] = trend.adx_pos(df['High'], df['Low'], df['Close'])


# Generate Trading Signals

For each TAs, a buy, hold and sell signal will be generated. They will bear the weight of 1, 0 and -1 respectively

In [24]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI
Time,,,,,,,,,,,,,,,
2024-07-15 21:17:00,63708.56,63718.00,63691.30,63691.31,9.93370,63682.8238,63496.71030,48.348510,11.292933,15.763101,-4.470168,-0.000386,18.687588,22.865224,23.566372
2024-07-15 21:18:00,63691.31,63691.31,63650.00,63657.01,10.87660,63683.7602,63499.36200,43.817910,5.832791,13.777039,-7.944249,-0.000767,18.311141,28.581771,21.819293
2024-07-15 21:19:00,63657.00,63660.00,63622.91,63626.74,9.82588,63684.2148,63502.08585,40.234668,-0.926270,10.836377,-11.762648,-0.000679,18.543554,31.555343,20.359870
2024-07-15 21:20:00,63626.74,63634.17,63626.73,63634.17,4.04671,63685.6932,63505.09360,41.499250,-5.618568,7.545388,-13.163957,-0.000563,18.759365,31.105890,20.069878
2024-07-15 21:21:00,63634.17,63634.17,63634.16,63634.17,3.41606,63687.1766,63507.91440,41.499250,-9.230841,4.190142,-13.420983,-0.000469,18.959762,31.105249,20.069464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17 19:53:00,64741.99,64748.00,64713.73,64713.74,10.98987,64629.0220,64383.63075,55.888568,26.832258,24.722764,2.109494,0.001141,18.006591,18.018383,27.784149
2024-07-17 19:54:00,64713.73,64739.05,64713.73,64720.97,14.63963,64632.3214,64386.42560,56.965299,25.843992,24.947010,0.896982,0.001098,18.243367,17.146133,26.439149
2024-07-17 19:55:00,64720.97,64720.97,64669.99,64683.47,13.46099,64635.5050,64388.89115,50.130360,21.783738,24.314355,-2.530617,-0.000013,16.979270,23.667701,23.927581


In [25]:
#SMA Golden Cross & Death Cross
sma_buy_cond = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) < df['SMA200'])
sma_sell_cond = (df['SMA50'] < df['SMA200']) & (df['SMA50'].shift(1) > df['SMA200'])
df['Signal_sma'] = np.where(sma_buy_cond, 1, np.where(sma_sell_cond, -1, 0))

In [26]:
# MACD Below 0 Cross
macd_buy_cond = (df['MACD_diff'] > 0) & (df['MACD_diff'].shift(1) < 0) & (df['MACD'] < 0) &(df['MACD_sig'] < 0)
macd_sell_cond = (df['MACD_diff'] < 0) & (df['MACD_diff'].shift(1) > 0) & (df['MACD'] > 0) &(df['MACD_sig'] > 0)
df['Signal_macd'] = np.where(macd_buy_cond, 1, np.where(macd_sell_cond, -1, 0))

In [27]:
# RSI Overbought & Oversold
rsi_buy_cond = (df['RSI'] > 30) & (df['RSI'].shift(1) < 30)
rsi_sell_cond = (df['RSI'] < 70) & (df['RSI'].shift(1) > 70)
df['Signal_rsi'] = np.where(rsi_buy_cond, 1, np.where(rsi_sell_cond, -1, 0))

In [28]:
# ADX Trend Confirmation
adx_buy_cond = (df['ADX'] > 25) & (df['+DI'] > df['-DI'])
adx_sell_cond = (df['ADX'] > 25) & (df['-DI'] > df['+DI'])
df['Signal_adx'] = np.where(adx_buy_cond, 1, np.where(adx_sell_cond, -1, 0))

In [29]:
# Momentum Trade
momentum_buy_cond = (df['Momentum'] > 0) & (df['Momentum'].shift(1) > 0) & (df['Momentum'].shift(2) > 0) & (df['Momentum'].shift(3) > 0) & (df['Momentum'].shift(4) > 0) \
                    & (df['Momentum'].shift(5) > 0) & (df['Momentum'].shift(6) > 0) & (df['Momentum'].shift(7) > 0) & (df['Momentum'].shift(8) > 0) & (df['Momentum'].shift(9) > 0)
momentum_sell_cond = (df['Momentum'] < 0) & (df['Momentum'].shift(1) < 0) & (df['Momentum'].shift(2) < 0) & (df['Momentum'].shift(3) < 0) & (df['Momentum'].shift(4) < 0) \
                    & (df['Momentum'].shift(5) < 0) & (df['Momentum'].shift(6) < 0) & (df['Momentum'].shift(7) < 0) & (df['Momentum'].shift(8) < 0) & (df['Momentum'].shift(9) < 0)
df['Signal_momentum'] = np.where(momentum_buy_cond, 1, np.where(momentum_sell_cond, -1, 0))

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd,Signal_rsi,Signal_adx,Signal_momentum
Time,,,,,,,,,,,,,,,,,,,,
2024-07-15 21:17:00,63708.56,63718.00,63691.30,63691.31,9.93370,63682.8238,63496.71030,48.348510,11.292933,15.763101,-4.470168,-0.000386,18.687588,22.865224,23.566372,0,0,0,0,0
2024-07-15 21:18:00,63691.31,63691.31,63650.00,63657.01,10.87660,63683.7602,63499.36200,43.817910,5.832791,13.777039,-7.944249,-0.000767,18.311141,28.581771,21.819293,0,0,0,0,0
2024-07-15 21:19:00,63657.00,63660.00,63622.91,63626.74,9.82588,63684.2148,63502.08585,40.234668,-0.926270,10.836377,-11.762648,-0.000679,18.543554,31.555343,20.359870,0,0,0,0,0
2024-07-15 21:20:00,63626.74,63634.17,63626.73,63634.17,4.04671,63685.6932,63505.09360,41.499250,-5.618568,7.545388,-13.163957,-0.000563,18.759365,31.105890,20.069878,0,0,0,0,0
2024-07-15 21:21:00,63634.17,63634.17,63634.16,63634.17,3.41606,63687.1766,63507.91440,41.499250,-9.230841,4.190142,-13.420983,-0.000469,18.959762,31.105249,20.069464,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17 19:53:00,64741.99,64748.00,64713.73,64713.74,10.98987,64629.0220,64383.63075,55.888568,26.832258,24.722764,2.109494,0.001141,18.006591,18.018383,27.784149,0,0,0,0,1
2024-07-17 19:54:00,64713.73,64739.05,64713.73,64720.97,14.63963,64632.3214,64386.42560,56.965299,25.843992,24.947010,0.896982,0.001098,18.243367,17.146133,26.439149,0,0,0,0,1
2024-07-17 19:55:00,64720.97,64720.97,64669.99,64683.47,13.46099,64635.5050,64388.89115,50.130360,21.783738,24.314355,-2.530617,-0.000013,16.979270,23.667701,23.927581,0,-1,0,0,0
